In [1]:
from live_utils import liveDownloader

%load_ext autoreload
%autoreload 2

In [2]:
keywords = {'bitcoin': ['bitcoin', 'BTC'], 'dashcoin': ['dashcoin', 'DASH', 'darkcoin'], 'dogecoin': ['dogecoin', 'DOGE'], 'ethereum': ['ethereum', 'ETH'], 'litecoin': ['litecoin', 'LTC'], 'ripple': ['ripple', 'XRP'], 'monero': ['monero', 'XMR'], 'stellar': ['stellar', 'STR']}

In [8]:
ld = liveDownloader(keywords)

In [9]:
df, userData = ld.collect()

[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin'

[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin'

Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), 

Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequen

[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin'

...

[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin'

[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin'

Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequen

Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), ('dogecoin', ['dogecoin', 'DOGE']), ('bitcoin', ['bitcoin', 'BTC']), ('litecoin', ['litecoin', 'LTC']), ('dashcoin', ['dashcoin', 'DASH', 'darkcoin']), ('monero', ['monero', 'XMR']), ('ripple', ['ripple', 'XRP']), ('ethereum', ['ethereum', 'ETH'])]
Got an exception. Error is: sequence item 0: expected str instance, tuple found
Error on line 123 TypeError sequence item 0: expected str instance, tuple found
[('stellar', ['stellar', 'STR']), 

KeyboardInterrupt: 

In [7]:
df

NameError: name 'df' is not defined